In [1]:
from IPython.display import clear_output

import re
import time

import requests
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

from tinydb import TinyDB, Query

In [2]:
db = TinyDB('data/custom_data/oddsportal_output.json')

In [3]:
tournaments = db.table('tournaments')
matches = db.table('matches')

In [4]:
query = Query()

In [5]:
"""
UPDATE THESE
"""
chrome_executable_path = r"C:\Users\Cristian Simionescu\Desktop\Master\Random\webdrivers\chromedriver.exe"
output_file_path = r"data/custom_data/oddsportal_output.txt"

In [6]:
base_url = "https://www.oddsportal.com"
start_url = "https://www.oddsportal.com/tennis/results/"
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Mobile Safari/537.36"
}

In [19]:
with requests.get(start_url, headers=headers) as r:
    links = re.findall(b'foo="f"\s*href="([^"]+)', r.content)
links = list(map(lambda x: x.decode(), links))

c = Chrome(executable_path=chrome_executable_path)

START_INDEX = 6 #0
END_INDEX = len(links)
for index, link in enumerate(links[START_INDEX:END_INDEX]):
    clear_output(wait=True)
    print("Tournament ID",index+START_INDEX)
    full_link = base_url + link
    link = link.lstrip('/').replace('tennis/', '')
    sep_index = link.find('/')
    if sep_index < 0:
        continue  # not found

    country = link[:sep_index]
    c.get(full_link)
    title = c.find_element_by_xpath('//h1').get_attribute('textContent')
    surface = re.search('\(\s*([a-z]+)\s*\)', title)
    if surface:
        surface = surface.group(1)
    else:
        surface = ""
    title = title.split(surface)[0].replace("(","").strip()
    table = c.find_element_by_id('tournamentTable')
    table = table.find_element_by_xpath('.//tbody')
    current_date = None
    # print(table.get_attribute("innerHTML"))
    for child in table.find_elements_by_xpath('.//tr'):
        if child.get_attribute('class') == "dark center":
            pass
        elif "nob-border" in child.get_attribute('class'):
            current_date = child.find_element_by_xpath('.//th/span').get_attribute('textContent').strip()
        elif 'deactivate' in child.get_attribute('class'):
            table_time = players = table_odds = info_val = ""
            odds_nowrp = list()
            for kid in child.find_elements_by_xpath('.//td'):
                if "table-time" in kid.get_attribute('class'):
                    table_time = kid.get_attribute('textContent')
                elif "table-participant" in kid.get_attribute('class'):
                    players = kid.get_attribute("textContent")
                    players = re.sub('<[^>]+>', '', players).split('-')
                    match_link = kid.find_element_by_css_selector("a").get_attribute("href")
                elif "table-odds" in kid.get_attribute('class'):
                    table_odds = kid.get_attribute('textContent')
                elif "odds-nowrp" in kid.get_attribute('class'):
                    odds_nowrp.append(kid.get_attribute('textContent'))
                elif "info-value" in kid.get_attribute('class'):
                    info_val = kid.get_attribute('textContent')
    print("Country:",country)
    print("Title:",title)
    print("Surface:",surface)
    print("Current_data:",current_date)
    print("Table_time:",table_time)
    print("Players:",players)
    print("Match_link:",match_link)
    print("Table_odds:",table_odds)
    print("Odds_nowrp:",odds_nowrp)
    print("Info_val:",info_val)
    time.sleep(5)
c.close()

Tournament ID 6
Country: argentina
Title: ATP Buenos Aires Doubles
Surface: clay
Current_data: 11 Feb 2019
Table_time: 16:45
Players: ['Cecchinato M./Lajovic D. ', ' Delbonis F./Duran G.']
Match_link: https://www.oddsportal.com/tennis/argentina/atp-buenos-aires-doubles/cecchinato-marco-lajovic-dusan-delbonis-federico-duran-guillermo-vHOW9MSS/
Table_odds: 2:1
Odds_nowrp: ['3.06', '1.37']
Info_val: 11


KeyboardInterrupt: 